In [1]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from webdriver_manager.core.os_manager import ChromeType
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service

In [2]:
# selenium 3
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from webdriver_manager.core.os_manager import ChromeType

import os

In [43]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select,WebDriverWait
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd
import numpy as np
from webdriver_manager.chrome import ChromeDriverManager

class TabelaResumoDataScraper:
    def __init__(self, setor_financeiro, options, service,acoes):
        self.setor_financeiro = setor_financeiro
        self.options = options
        self.service = service
        self.acoes = acoes

    def navegador_get(self, acao):
        navegador = webdriver.Chrome(service=self.service, options=self.options)
        navegador.get(f'https://www.investsite.com.br/principais_indicadores.php?cod_negociacao={acao}')
        return navegador

    def obter_datas(self, navegador):
        while True:
            try:
                elementos_data = navegador.find_elements(By.XPATH, '//*[@id="tabela_resumo_empresa_bp"]/thead/tr/th[2]/select')
                datas = [elemento.text for elemento in elementos_data]
                if len(datas) == 1:
                    string_de_datas = datas[0]
                    datas = string_de_datas.split('\n')
                return datas
            except StaleElementReferenceException:
                continue

    def obter_dados_tabela(self, navegador, data):
        while True:
            try:
                select_element = navegador.find_element(By.XPATH, '/html/body/div[1]/div[4]/div[2]/div[2]/div[3]/table/thead/tr/th[2]/select')
                select = Select(select_element)
                select.select_by_visible_text(data)

                tabela = navegador.find_element(By.ID, 'pagina_empresa_bp')
                linhas = tabela.find_elements(By.TAG_NAME, 'tr')

                lista_resumo_balanco = []
                for linha in linhas:
                    celulas = linha.find_elements(By.TAG_NAME, 'td')
                    for celula in celulas:
                        lista_resumo_balanco.append(celula.text)

                return lista_resumo_balanco
            except StaleElementReferenceException:
                continue

    def coletar_dados_financeiros(self, navegador, datas):
        dados = {
            'datas': datas,
            'caixa_equivalentes_caixa': [np.nan] * len(datas),
            'ativo_total': [],
            'divida_curto_prazo': [np.nan] * len(datas),
            'divida_longo_prazo': [np.nan] * len(datas),
            'divida_bruta': [np.nan] * len(datas),
            'divida_liquida': [np.nan] * len(datas),
            'patrimonio_liquido': [],
            'valor_patrimonial_acao': [],
            'acoes_ordinarias': [],
            'acoes_preferenciais': [],
            'total': []
        }
        for data in datas:
            while True:
                lista_resumo_balanco = self.obter_dados_tabela(navegador=navegador, datas=data)
                if lista_resumo_balanco:
                    break
            dados['ativo_total'].append(lista_resumo_balanco[1].replace('R$','').replace(',','.').strip())
            dados['patrimonio_liquido'].append(lista_resumo_balanco[3].replace('R$','').replace(',','.').strip())
            dados['valor_patrimonial_acao'].append(lista_resumo_balanco[5].replace('R$','').replace(',','.').strip())
            dados['acoes_ordinarias'].append(lista_resumo_balanco[7].replace('R$','').replace(',','.').strip())
            dados['acoes_preferenciais'].append(lista_resumo_balanco[9].replace('R$','').replace(',','.').strip())
            dados['total'].append(lista_resumo_balanco[11].replace('R$','').replace(',','.').strip())
            
        df_resumo_balanco = pd.DataFrame(dados)
        
        return df_resumo_balanco
            
    def coletar_dados_nao_financeiros(self,navegador, datas):
        dados = {
            'datas': datas,
            'caixa_equivalentes_caixa': [],
            'ativo_total': [],
            'divida_curto_prazo': [],
            'divida_longo_prazo': [],
            'divida_bruta': [],
            'divida_liquida': [],
            'patrimonio_liquido': [],
            'valor_patrimonial_acao': [],
            'acoes_ordinarias': [],
            'acoes_preferenciais': [],
            'total': []
        }

        for data in datas:
            while True:
                lista_resumo_balanco = self.obter_dados_tabela(navegador=navegador, data=data)
                if lista_resumo_balanco:
                    break
            dados['caixa_equivalentes_caixa'].append(lista_resumo_balanco[1].replace('R$','').replace(',','.').strip())
            dados['ativo_total'].append(lista_resumo_balanco[3].replace('R$','').replace(',','.').strip())
            dados['divida_curto_prazo'].append(lista_resumo_balanco[5].replace('R$','').replace(',','.').strip())
            dados['divida_longo_prazo'].append(lista_resumo_balanco[7].replace('R$','').replace(',','.').strip())
            dados['divida_bruta'].append(lista_resumo_balanco[9].replace('R$','').replace(',','.').strip())
            dados['divida_liquida'].append(lista_resumo_balanco[11].replace('R$','').replace(',','.').strip())
            dados['patrimonio_liquido'].append(lista_resumo_balanco[13].replace('R$','').replace(',','.').strip())
            dados['valor_patrimonial_acao'].append(lista_resumo_balanco[15].replace('R$','').replace(',','.').strip())
            dados['acoes_ordinarias'].append(lista_resumo_balanco[17].replace('R$','').replace(',','.').strip())
            dados['acoes_preferenciais'].append(lista_resumo_balanco[19].replace('R$','').replace(',','.').strip())
            dados['total'].append(lista_resumo_balanco[21].replace('R$','').replace(',','.').strip())
            
        df_resumo_balanco = pd.DataFrame(dados)
            
        return df_resumo_balanco
    
    def rodar_acoes(self):
        iteracao = 0
        lista_dataframes = []
        for acao in self.acoes:
            start_time = time.time()
            navegador = self.navegador_get(acao=acao)
            
            datas = self.obter_datas(navegador=navegador)
            
            if acao in self.setor_financeiro:
                df_resumo_balanco = self.coletar_dados_financeiros(navegador=navegador, data=datas)  
                colunas = ['ativo_total', 'patrimonio_liquido']
                for col in colunas:
                    df_resumo_balanco[col] = df_resumo_balanco[col].apply(self.converter_valor)
            else:
                df_resumo_balanco = self.coletar_dados_nao_financeiros(navegador=navegador, datas=datas)  
                colunas = ['caixa_equivalentes_caixa', 'ativo_total', 'divida_liquida','divida_curto_prazo', 'divida_longo_prazo', 'divida_bruta', 'patrimonio_liquido']
                for col in colunas:
                    df_resumo_balanco[col] = df_resumo_balanco[col].apply(self.converter_valor)
                  
            df_resumo_balanco['tic'] = acao
            lista_dataframes.append(df_resumo_balanco)

            iteracao += 1
            
            if iteracao % 5 == 0:
                self.salvar_dados(lista_dataframes=lista_dataframes)
                
            end_time = time.time()

            # Calcula o tempo decorrido em segundos
            execution_time_seconds = end_time - start_time

            # Calcula o tempo decorrido em minutos
            execution_time_minutes = execution_time_seconds / 60

            # Imprime o tempo de execução em minutos
            print(f"Tempo de execução da ação {acao}: {execution_time_minutes:.2f} minutos")
                
        return pd.concat(lista_dataframes)

    def salvar_dados(self,lista_dataframes):
        pd.concat(lista_dataframes).to_csv('dados/resumo_balanco.csv')

    def converter_valor(self,valor):
        if isinstance(valor, str):
            if valor.endswith('T'):
                return float(valor.replace('T', '').replace(' ', '')) * 1_000_000
            elif valor.endswith('B'):
                return float(valor.replace('B', '').replace(' ', '')) * 1_000
            elif valor.endswith('M'):
                return float(valor.replace('M', '').replace(' ', '')) * 1
            elif valor.endswith('K'):
                return float(valor.replace('K', '').replace(' ', '')) * 0.001
            elif valor == 'NA':
                return float('nan')
        else:
            return valor

In [44]:
codigos_ibovespa = [
    'ABEV3', 'AZUL4', 'B3SA3', 'BBAS3', 'BBDC3', 'BBDC4', 'BBSE3', 'BEEF3', 'BPAC11', 'BRAP4',]

In [45]:
# Uso da classe
setor_financeiro = {'BBAS3', 'BBDC3', 'BBDC4', 'BBSE3', 'ITUB4', 'BPAC11', 'ITUB4', 'SANB11', 'IRBR3'}

chrome_driver_path = "/usr/bin/chromedriver"
service = Service(executable_path=chrome_driver_path)

options = Options()
options.add_argument("--headless")  # Executar em modo headless
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--disable-dev-shm-usage")  # Reduz o uso de memória
options.add_argument("--disable-gpu")  # Desativar o uso de GPU
options.add_argument("--window-size=1920,1080")  # Definir o tamanho da janela

scraper = TabelaResumoDataScraper(setor_financeiro, options, service,acoes=codigos_ibovespa)

dados = scraper.rodar_acoes()

UnexpectedAlertPresentException: Alert Text: Erro. Por favor tente novamente.
Message: unexpected alert open: {Alert text : Erro. Por favor tente novamente.}
  (Session info: chrome-headless-shell=127.0.6533.72)
Stacktrace:
#0 0x561e031cc6ba <unknown>
#1 0x561e02e9c730 <unknown>
#2 0x561e02f2dae1 <unknown>
#3 0x561e02f0e7f3 <unknown>
#4 0x561e02edeec9 <unknown>
#5 0x561e02edf91e <unknown>
#6 0x561e031929eb <unknown>
#7 0x561e03196972 <unknown>
#8 0x561e0317fe15 <unknown>
#9 0x561e03197502 <unknown>
#10 0x561e03164d2f <unknown>
#11 0x561e031bb578 <unknown>
#12 0x561e031bb750 <unknown>
#13 0x561e031cb48c <unknown>
#14 0x7efc33020a94 <unknown>
#15 0x7efc330adc3c <unknown>


In [39]:
dados

,datas,caixa_equivalentes_caixa,ativo_total,divida_curto_prazo,divida_longo_prazo,divida_bruta,divida_liquida,patrimonio_liquido,valor_patrimonial_acao,acoes_ordinarias,acoes_preferenciais,total,tic
0,31/03/2024,12840.00,133820.0,1650.00,2170.00,3820.0,-9030.00,86590.00,5.50,15.757.657.000,0,15.757.657.000,ABEV3
1,31/12/2023,16060.00,132640.0,1300.00,2200.00,3500.0,-12560.00,78970.00,5.01,15.753.833.000,0,15.753.833.000,ABEV3
2,30/09/2023,17410.00,137910.0,1240.00,2480.00,3720.0,-13700.00,90250.00,5.73,15.753.833.000,0,15.753.833.000,ABEV3
3,30/06/2023,12120.00,133290.0,1320.00,2670.00,3990.0,-8120.00,85070.00,5.40,15.753.833.000,0,15.753.833.000,ABEV3
4,31/03/2023,12210.00,135470.0,1140.00,2650.00,3790.0,-8420.00,85000.00,5.40,15.753.833.000,0,15.753.833.000,ABEV3
5,31/12/2022,14930.00,137960.0,982.57,2790.00,3770.0,-11160.00,81960.00,5.20,15.750.217.000,0,15.750.217.000,ABEV3
6,30/09/2022,17710.00,142060.0,761.51,2220.00,2980.0,-14730.00,90180.00,5.73,15.750.217.000,0,15.750.217.000,ABEV3
7,30/06/2022,14130.00,136630.0,894.36,2300.00,3200.0,-10930.00,86260.00,5.48,15.750.217.000,0,15.750.217.000,ABEV3
8,31/03/2022,12890.00,127400.0,815.86,2170.00,2990.0,-9900.00,78920.00,5.01,15.750.217.000,0,15.750.217.000,ABEV3
9,31/12/2021,16630.00,138600.0,847.12,2250.00,3100.0,-13530.00,82640.00,5.25,15.744.452.000,0,15.744.452.000,ABEV3


In [ ]:
from selenium.webdriver.chrome.service import Service
refazer_comeco = True
chrome_driver_path = "/usr/bin/chromedriver"
service = Service(executable_path=chrome_driver_path)

options = Options()
options.add_argument("--headless")  # Executar em modo headless
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--disable-dev-shm-usage")  # Reduz o uso de memória
options.add_argument("--disable-gpu")  # Desativar o uso de GPU
options.add_argument("--window-size=1920,1080")  # Definir o tamanho da janela

codigos_ibovespa = [
    'ABEV3', 'AZUL4', 'B3SA3', 'BBAS3', 'BBDC3', 'BBDC4', 'BBSE3', 'BEEF3', 'BPAC11', 'BRAP4',
    #'VVBR3', 'BTOW3','ENBR3',GNDI3,IGTA3
    'BRFS3', 'BRKM5',  'CCRO3', 'CIEL3', 'CMIG4', 'COGN3', 'CPFE3',
    'CRFB3', 'CSAN3', 'CSNA3', 'CVCB3', 'CYRE3', 'ECOR3', 'EGIE3', 'ELET3', 'ELET6', 'EMBR3',
     'ENGI11', 'EQTL3', 'EZTC3', 'FLRY3', 'GGBR4',  'GOAU4', 'GOLL4', 'HAPV3',
     'HYPE3', 'IRBR3', 'ITSA4', 'ITUB4', 'JBSS3', 'KLBN11', 'LREN3',
    'MGLU3', 'MRFG3', 'MRVE3', 'MULT3', 'NTCO3', 'PCAR3', 'PETR3', 'PETR4', 'PRIO3', 'QUAL3',
    'RADL3', 'RAIL3', 'RENT3', 'SANB11', 'SBSP3','SUZB3', 'TAEE11', 'TIMS3', 'TOTS3',
    'UGPA3', 'USIM5', 'VALE3', 'VIVT3', 'WEGE3', 'WIZC3','YDUQ3','SOMA3','BPAC11','MDIA3'
]